In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot  as plt
import gensim

from nltk import word_tokenize
from nltk.corpus import stopwords
import string

from gensim.models import CoherenceModel, LdaModel, LsiModel, HdpModel
from gensim.models.wrappers import LdaMallet
from gensim.corpora import Dictionary
from gensim import corpora

import pyLDAvis.gensim
pyLDAvis.enable_notebook()


import os, re, operator, warnings
warnings.filterwarnings('ignore')  # Let's not pay heed to them right now
%matplotlib inline

c:\users\jmilanes\appdata\local\programs\python\python37-32\lib\site-packages\gensim\utils.py:1209: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
c:\users\jmilanes\appdata\local\programs\python\python37-32\lib\site-packages\funcy\colls.py:6: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Mapping, Set, Iterable, Iterator, defaultdict
c:\users\jmilanes\appdata\local\programs\python\python37-32\lib\site-packages\funcy\colls.py:6: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Mapping, Set, Iterable, Iterator, defaultdict
c:\users\jmilanes\appdata\local\programs\python\python37-32\lib\site-packages\funcy\seqs.py:3: DeprecationWarning: Using or im

In [2]:
df = pd.read_pickle('mejor_lematizado.pkl')
df

,Unnamed: 0,expositor,texto,fecha,apellido,nombre,fecha_exp,profesion,profesionresumen,rol,genero,posicion,lematizado
0,1,Bergallo,Es un honor para mi acompañarlos con el fin d...,2018.04.10,Bergallo,Paola,2018.04.10,abogado,abogado/a,profesora de Derecho en las universidades Di T...,mujer,a favor,ser uno honor para mi acompañarlos con el fin ...
1,2,Bergallo,Estos son mas ejemplos de como no hemos tomad...,2018.04.10,Bergallo,Paola,2018.04.10,abogado,abogado/a,profesora de Derecho en las universidades Di T...,mujer,a favor,este ser mas ejemplo de como no haber tomar en...
2,3,Gil Dominguez,"Buenos dias, diputados y diputadas. Muchas gr...",2018.04.10,Gil Dominguez,Andres,NaN,NaN,NaN,NaN,varon,a favor,"bueno dias , diputado y diputado . mucho graci..."
3,4,Alanis,"En primer termino, quiero expresar mi agradec...",2018.04.10,Alanis,Marta,NaN,NaN,NaN,NaN,mujer,a favor,"en 1 termino , querer expresar mi agradecimien..."
4,5,Romero,Agradezco la oportunidad y la atencion que no...,2018.04.10,Romero,Mariana,2018.04.10,medica,medico/a,investigadora del Cedes y el Conicet,mujer,a favor,agradecer el oportunidad y el atencion que nos...
5,6,Alegre,No hay honor mayor para un ciudadano que ser ...,2018.04.10,Alegre,Marcelo,2018.04.10,abogado,abogado/a,abogado especialista en etica aplicada.,varon,a favor,no hay honor mayor para uno ciudadano que ser ...
6,7,Miyensky,Buenos dias. Ante todo quiero agradecer a los...,2018.04.10,Miyenski,Nelly,NaN,NaN,NaN,NaN,mujer,a favor,bueno dias . ante todo querer agradecer a el d...
7,8,Chellier,"Buenos dias, diputados y diputadas. Desde el ...",2018.04.10,Chellier,Gaston,2018.04.10,NaN,abogado/a,director ejecutivo del CELS,varon,a favor,"bueno dias , diputado y diputado . desde el ce..."
8,9,Vazquez,En primer termino quiero agradecer este espac...,2018.04.10,Vazquez,Sandra,2018.04.10,ginecologa,medico/a,ginecologa del Hospital Argerich. directora de...,mujer,a favor,en 1 termino querer agradecer este espacio par...
9,10,Caruana,Gracias a todos los diputados por haberme inv...,2018.04.10,Caruana,Leonardo,2018.04.10,Medico,medico/a,secretario de Salud Publica de la Municipalida...,varon,a favor,gracia a todo el diputado por haberme invitado...


In [3]:
stopwords = stopwords.words('spanish')+list(string.punctuation)+list("\'\'")

In [4]:
def procesar_texto(text, stopwords=[]):
    text = word_tokenize(text.lower())
    return [tok for tok in text if tok not in stopwords]

In [5]:
textoslematizados = [procesar_texto(x,stopwords) for x in df.lematizado]
textosfavor = [procesar_texto(x,stopwords) for x in df.lematizado[df.posicion=='a favor']]
textoscontra = [procesar_texto(x,stopwords) for x in df.lematizado[df.posicion=='en contra']]

In [6]:
dictionarylem = corpora.Dictionary(textoslematizados)
print(len(dictionarylem))
dictionarylem.filter_extremes()
print(len(dictionarylem))
corpuslematizado = [dictionarylem.doc2bow(text) for text in textoslematizados]
ldamodel = LdaModel(corpus=corpuslematizado, num_topics=10, id2word=dictionarylem, iterations=4000, passes=100)
ldamodel.print_topics(10, 10)

20790
5251


[(0,
  '0.014*"querer" + 0.013*"hijo" + 0.010*"año" + 0.008*"hablar" + 0.007*"abortar" + 0.007*"dar" + 0.007*"ver" + 0.007*"madre" + 0.006*"niño" + 0.006*"estan"'),
 (1,
  '0.024*"humano" + 0.015*"persona" + 0.012*"madre" + 0.010*"niño" + 0.009*"nacer" + 0.009*"embrion" + 0.007*"deber" + 0.007*"momento" + 0.006*"embarazo" + 0.006*"concepcion"'),
 (2,
  '0.015*"sexual" + 0.012*"educacion" + 0.010*"ley" + 0.010*"año" + 0.009*"nacional" + 0.009*"legal" + 0.008*"diputado" + 0.008*"provincia" + 0.008*"hoy" + 0.007*"feminista"'),
 (3,
  '0.019*"materno" + 0.017*"mortalidad" + 0.017*"ciento" + 0.017*"muerte" + 0.014*"salud" + 0.013*"año" + 0.013*"paises" + 0.008*"argentina" + 0.008*"causa" + 0.008*"tasa"'),
 (4,
  '0.024*"semana" + 0.013*"ver" + 0.010*"embarazo" + 0.010*"bebe" + 0.009*"medico" + 0.008*"feto" + 0.007*"realizar" + 0.007*"dos" + 0.007*"1" + 0.007*"paciente"'),
 (5,
  '0.016*"penal" + 0.012*"idea" + 0.012*"pena" + 0.009*"ver" + 0.008*"querer" + 0.007*"objecion" + 0.007*"persona" 

In [7]:
dictionaryfavor = corpora.Dictionary(textosfavor)
print(len(dictionaryfavor))
dictionaryfavor.filter_extremes()
print(len(dictionaryfavor))
corpusfavor = [dictionaryfavor.doc2bow(text) for text in textosfavor]
ldamodelfavor = LdaModel(corpus=corpusfavor, num_topics=10, id2word=dictionaryfavor, iterations=4000, passes=100)
ldamodelfavor.print_topics(10, 10)

13774
3212


[(0,
  '0.010*"abortar" + 0.009*"cuerpo" + 0.007*"decidir" + 0.007*"hablar" + 0.007*"hijo" + 0.006*"ley" + 0.006*"diputado" + 0.006*"legal" + 0.006*"hoy" + 0.005*"seguro"'),
 (1,
  '0.013*"persona" + 0.010*"social" + 0.009*"cuerpo" + 0.009*"deber" + 0.008*"politica" + 0.008*"niño" + 0.007*"sociedad" + 0.007*"nacer" + 0.006*"existir" + 0.006*"solo"'),
 (2,
  '0.033*"sexual" + 0.021*"educacion" + 0.014*"integral" + 0.011*"ley" + 0.008*"violencia" + 0.008*"provincia" + 0.008*"reproductivo" + 0.007*"adolescente" + 0.007*"anticonceptivo" + 0.006*"niño"'),
 (3,
  '0.014*"religioso" + 0.012*"ley" + 0.010*"conciencia" + 0.010*"deber" + 0.010*"proyecto" + 0.009*"diputado" + 0.009*"debate" + 0.008*"señor" + 0.007*"iglesia" + 0.006*"laico"'),
 (4,
  '0.010*"hospital" + 0.008*"atencion" + 0.008*"profesional" + 0.008*"legal" + 0.007*"seguro" + 0.007*"acceso" + 0.006*"interrupcion" + 0.006*"sistema" + 0.006*"centro" + 0.006*"hijo"'),
 (5,
  '0.011*"decidir" + 0.011*"persona" + 0.011*"deber" + 0.009*

In [8]:
dictionarycontra = corpora.Dictionary(textoscontra)
print(len(dictionarycontra))
dictionarycontra.filter_extremes()
print(len(dictionarycontra))
corpuscontra = [dictionarycontra.doc2bow(text) for text in textoscontra]
ldamodelcontra = LdaModel(corpus=corpuscontra, num_topics=5, id2word=dictionarycontra, iterations=4000, passes=100)
ldamodelcontra.print_topics(5, 10)

14803
3447


[(0,
  '0.010*"ver" + 0.009*"semana" + 0.008*"embarazo" + 0.008*"humano" + 0.007*"ir" + 0.007*"madre" + 0.006*"tambien" + 0.006*"embrion" + 0.006*"deber" + 0.005*"niño"'),
 (1,
  '0.014*"hijo" + 0.012*"ir" + 0.012*"querer" + 0.010*"año" + 0.008*"abortar" + 0.007*"hablar" + 0.007*"madre" + 0.007*"dar" + 0.007*"vez" + 0.007*"embarazo"'),
 (2,
  '0.039*"derecho" + 0.016*"humano" + 0.014*"persona" + 0.013*"niño" + 0.010*"deber" + 0.009*"ley" + 0.008*"nacer" + 0.008*"articulo" + 0.008*"caso" + 0.007*"proyecto"'),
 (3,
  '0.015*"salud" + 0.014*"materno" + 0.013*"muerte" + 0.011*"mortalidad" + 0.010*"ciento" + 0.009*"año" + 0.008*"paises" + 0.008*"embarazo" + 0.007*"causa" + 0.006*"argentina"'),
 (4,
  '0.033*"humano" + 0.011*"persona" + 0.010*"embrion" + 0.009*"momento" + 0.008*"ir" + 0.007*"desarrollo" + 0.007*"tambien" + 0.006*"hombre" + 0.006*"madre" + 0.006*"hablar"')]

In [9]:
print(ldamodelcontra)
print(dictionarycontra)
print(type(dictionarycontra))
print(type(corpuslematizado))

LdaModel(num_terms=3447, num_topics=5, decay=0.5, chunksize=2000)
Dictionary(3447 unique tokens: ['palabra', 'pedir', "'la", 'absolutamente', 'aca']...)
<class 'gensim.corpora.dictionary.Dictionary'>
<class 'list'>


In [10]:
pyLDAvis.gensim.prepare(ldamodel, corpuslematizado, dictionarylem)

TypeError: Could not operate array([453.,  44., 403., ..., 508., 452., 160.]) with block values unexpected indent (test_numexpr.py, line 1279)

In [11]:
pyLDAvis.gensim.prepare(ldamodel, corpuslematizado, dictionarylem,  mds="tsne")

TypeError: Could not operate array([453.,  44., 403., ..., 508., 452., 160.]) with block values unexpected indent (test_numexpr.py, line 1279)

In [88]:
ldatopics = [[word for word, prob in topic] for topicid, topic in ldamodelcontra.show_topics(formatted=False)]